In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

When I first explored this dataset, i found it so clumsy,and so messy, that I decided to cut it. 

The data was really unbalanced : it had a few tweets from 2009, most of the tweets are in english (90%) but those handful foreign tweets do mess up with the analysis.

Here is the code to cut this dataset to : 
. data from 2017-01-01
. tweets only in English language

In [ ]:
from time import time
import pandas as pd
import numpy as np
import re
import sys
import csv
csv.field_size_limit(sys.maxsize)

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns


In [ ]:
# Reading the dataset with no columns titles and with latin encoding 
df_raw = pd.read_csv('/kaggle/input/bitcoin-tweets-20160101-to-20190329/tweets.csv', delimiter=';', skiprows=0, lineterminator='\n' )
df_raw.head()

In [ ]:
 # Rename columns
df_raw.columns = ["id", "user", "fullname", "url", "timestamp", "replies","likes","retweets","text"]

# Show the first 5 rows of the dataframe.
# You can specify the number of rows to be shown as follows: df_raw.head(10)
df_raw.head()

In [ ]:
# Ommiting every column except for the text and the date, as we won't need any of the other information
df = df_raw[['timestamp','text']]
df.sample(5)

In [ ]:
#Create split by dates 
df.timestamp.dtypes

In [ ]:
df['date'] = pd.to_datetime(df['timestamp'],format= '%Y-%m-%d').dt.date
df['date']

In [ ]:
#new df is now SORTED by DATE
df = df.sort_values(by='date')
df.head()

In [ ]:
df = df.set_index('date')
df.head()

In [ ]:
# df.date.year > '2017'
startdate = pd.to_datetime("2017-01-01").date()
df = df.loc[startdate:]
df.tail()

Now that we have cleant the dates of our data => let s analyze how many tweets are in which language, and if unbalanced, we will only keep the tweets in ENglish

In [ ]:
pip install whatthelang

In [ ]:
from whatthelang import WhatTheLang
wtl = WhatTheLang()
result = [wtl.predict_lang(row) for row in df['text']]
df['lang'] = result
df.head()

In [ ]:
#ATTENTION, we will take only text, date and lang column FOR THE texts in ENglish and below DATES starting 2017 jan
en_df = df[df["lang"] == 'en']
en_df.sample(10, random_state = 5)

In [ ]:
# checking for null values, if any
en_df.isnull().sum()

In [ ]:
#ditching all row when text is null, as need text for analysis
en_df.dropna(how='any', inplace=True)
en_df.sample(3)

In [ ]:
en_df.info()

In [ ]:
#export to new CSV as it kept crashing my google colab otherwise
# en_df = reset_index(inplace=True)
en_df.to_csv('/kaggle/input/bitcoin-tweets-20160101-to-20190329/tweetsENdates.csv')

Now that we have something less clumsy to work on, I will work on the data preprocessing before modeling 
Here is the step 2 [suite...](http://www.kaggle.com/leticehs/nlp-bitcoin-analysis-2-preprocessing)